In [1]:
from neo4j import GraphDatabase
import networkx as nx
from networkx.readwrite import json_graph
import json
import matplotlib.pyplot as plt
import random
from json import JSONEncoder
import datetime
from neo4j.time import DateTime

In [2]:
uri = "neo4j+s://f070b0f3.databases.neo4j.io"
user = "neo4j"
password = "F5jPhWhvl2l83WpnCyS3hReVJQ2usxtZhokUW3Dg4C8"

driver = GraphDatabase.driver(uri, auth=(user, password))

def fetch_data(tx):
    result = tx.run("MATCH (n)-[r]->(m) RETURN n, r, m")
    return [(record["n"], record["r"], record["m"]) for record in result]

with driver.session() as session:
    graph_data = session.read_transaction(fetch_data)


C:\Users\vijmr\AppData\Local\Temp/ipykernel_40080/2115641671.py:12: DeprecationWarning: read_transaction has been renamed to execute_read
  graph_data = session.read_transaction(fetch_data)


In [3]:
# Initialize a directed multigraph
G = nx.MultiDiGraph()

# Add nodes and edges to the graph
for n, r, m in graph_data:
    # Assuming 'id' is a unique identifier for your nodes
    n_id = n.get("id")
    m_id = m.get("id")
    
    # Add nodes
    if not G.has_node(n_id):
        G.add_node(n_id, **dict(n.items()))
    if not G.has_node(m_id):
        G.add_node(m_id, **dict(m.items()))
    
    # Add edge
    G.add_edge(n_id, m_id, **dict(r.items()))

In [4]:
data = json_graph.node_link_data(G)


In [5]:
data

{'directed': True,
 'multigraph': True,
 'graph': {},
 'nodes': [{'gender': 'F',
   'reportDate': neo4j.time.DateTime(2021, 8, 27, 0, 0, 0, 0, tzinfo=<UTC>),
   'ageUnit': 'YR',
   'reporterOccupation': 'Physician',
   'id': 0,
   'primaryid': 100654764,
   'age': 56.0,
   'eventDate': neo4j.time.DateTime(2014, 3, 25, 0, 0, 0, 0, tzinfo=<UTC>)},
  {'id': 142, 'ageGroup': 'Adult'},
  {'description': 'Pulmonary embolism', 'id': 4454},
  {'name': 'CLONAZEPAM', 'id': 7162, 'primarySubstabce': 'CLONAZEPAM'},
  {'name': 'DEXAMETHASONE', 'id': 7163, 'primarySubstabce': 'DEXAMETHASONE'},
  {'name': 'KEPPRA', 'id': 7164, 'primarySubstabce': 'LEVETIRACETAM'},
  {'name': 'LISINOPRIL', 'id': 7165, 'primarySubstabce': 'LISINOPRIL'},
  {'name': 'METOPROLOL SUCCINATE',
   'id': 7166,
   'primarySubstabce': 'METOPROLOL SUCCINATE'},
  {'name': 'ONDANSETRON', 'id': 7167, 'primarySubstabce': 'ONDANSETRON'},
  {'name': 'VIMPAT', 'id': 7168, 'primarySubstabce': 'LACOSAMIDE'},
  {'name': 'SENOKOT', 'id': 71

In [6]:
def custom_handler(obj):
    if isinstance(obj, DateTime):
        # Convert neo4j.time.DateTime to a string or another serializable format
        return obj.iso_format()  # Adjust this method call based on Neo4j's DateTime API
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

# Assuming 'data' is your graph data prepared for serialization
with open('graphData.json', 'w') as f:
    json.dump(data, f, indent=4, default=custom_handler)
